Here to try to finetune the CNN networks with my own PC (single 1080ti) on the tiny imagenet dataset
First we download the data

In [1]:
import os
import requests
import zipfile

# Retrieve the data
if not os.path.exists(os.path.join('data','tiny-imagenet-200.zip')):
    url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.mkdir('data')
    
    # Write to a file
    with open(os.path.join('data','tiny-imagenet-200.zip'), 'wb') as f:
        f.write(r.content)
else:
    print("The zip file already exists.")
    
if not os.path.exists(os.path.join('data', 'tiny-imagenet-200')):
    with zipfile.ZipFile(os.path.join('data','tiny-imagenet-200.zip'), 'r') as zip_ref:
        zip_ref.extractall('data')
else:
    print("The extracted data already exists")

The zip file already exists.
The extracted data already exists


Chekcing the setups and importing related packages

In [2]:
from functools import partial
import tensorflow as tf
#import tensorflow_hub as hub
import requests
import zipfile
import requests
import os
import time
import pandas as pd
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AvgPool2D, Dense, Concatenate, Flatten, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
import numpy as np
from PIL import Image
import tensorflow.keras.backend as K
import pickle
from tensorflow.keras.models import load_model, Model
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed, using my birth year
random_seed = 1997
fix_random_seed(random_seed)

print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.10.0


Functions for parsing teh dataset

In [3]:
def get_test_labels_df(test_labels_path):
    """ Reading the test data labels for all files in the test set as a data frame """
    test_df = pd.read_csv(test_labels_path, sep='\t', index_col=None, header=None)
    test_df = test_df.iloc[:,[0,1]].rename({0:"filename", 1:"class"}, axis=1)
    return test_df

def get_train_valid_test_data_generators(batch_size, target_size):
    """ Get the training/validation/testing data generators """
    
    # Defining a data-augmenting image data generator and a standard image data generator
    image_gen_aug = ImageDataGenerator(
        samplewise_center=False, rotation_range=30, width_shift_range=0.2,
        height_shift_range=0.2, brightness_range=(0.5,1.5), shear_range=5, 
        zoom_range=0.2, horizontal_flip=True, fill_mode='constant', cval=127.5, 
        validation_split=0.1
    )
    image_gen = ImageDataGenerator(samplewise_center=False)
    
    # Define a training data generator
    partial_flow_func = partial(
        image_gen_aug.flow_from_directory, 
        directory=os.path.join('data','tiny-imagenet-200', 'train'), 
        target_size=target_size, classes=None,
        class_mode='categorical', batch_size=batch_size, 
        shuffle=True, seed=random_seed)
    
    # Get the training data subset
    train_gen = partial_flow_func(subset='training')
    # Get the validation data subset
    valid_gen = partial_flow_func(subset='validation')    

    # Defining the test data generator
    test_df = get_test_labels_df(os.path.join('data','tiny-imagenet-200',  'val', 'val_annotations.txt'))
    test_gen = image_gen.flow_from_dataframe(
        test_df, directory=os.path.join('data','tiny-imagenet-200',  'val', 'images'), target_size=target_size, classes=None,
        class_mode='categorical', batch_size=batch_size, shuffle=False
    )
    return train_gen, valid_gen, test_gen


batch_size = 32
target_size = (224,224)
# Getting the train,valid, test data generators
train_gen, valid_gen, test_gen = get_train_valid_test_data_generators(batch_size, target_size)
# Modifying the data generators to fit the model targets
train_gen_inceptionV3, valid_gen_inceptionV3, test_gen_inceptionV3 = get_train_valid_test_data_generators(batch_size,(299,299))

with open(os.path.join('data','class_indices'), 'wb') as f:
    pickle.dump(train_gen.class_indices, f)

Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.
Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


The models we are finetuning are:
1. VGG family: 16 and 19
2. ResNet family: 50v2 and 101 v2
3. InceptionV3
From a preliminary run of the code, I expect difficulty with InceptionV3. My GPU has limited (11GB) memory space and it's likely to eat it all up.
None the less, all network are trained with 15 epoch, then 50 epochs, then 100 epoch, or till an error forces a stop.
Accuracy and loss score are either calculated if the training is completed, or taken using the last finished epoch's value

In [4]:
from tensorflow.keras.applications import VGG19,ResNet50V2,InceptionV3,VGG16,ResNet101V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D

def create_inceptionv3_model(input_shape=(299, 299, 3), num_classes=200):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

inceptionv3_model = create_inceptionv3_model()
inceptionv3_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_resnet50v2_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

resnet50v2_model = create_resnet50v2_model()
resnet50v2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_vgg19_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    x = Flatten()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

vgg19_model = create_vgg19_model()
vgg19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_ResNet101V2_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = ResNet101V2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

resnet101V2_model = create_ResNet101V2_model()
resnet101V2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
def create_vgg16_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = Flatten()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

vgg16_model = create_vgg16_model()
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#vgg16_model.summary()
#resnet101V2_model.summary()
#vgg19_model.summary()
#resnet50v2_model.summary()
#inceptionv3_model.summary()

models = [(vgg16_model,"vgg_16_"),(vgg19_model,"vgg_19_"),(resnet50v2_model,"resnet_50_"),(resnet101V2_model,"resnet_101_"),(inceptionv3_model,"inception_v3")]

Helper functions for early stopping and changing learning rates

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ReduceLROnPlateau
es_callback = EarlyStopping(monitor='val_loss', patience=25)
lr_callback = ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto'
)
def get_steps_per_epoch(n_data, batch_size):
    """ Given the data size and batch size, gives the number of steps to travers the full dataset """
    if n_data%batch_size==0:
        return int(n_data/batch_size)
    else:
        return int(n_data*1.0/batch_size)+1

Function for finetuning the models, and getting the test result

In [6]:
def train_test_model(model,name,n_epochs):
    print("Training model:",name[:-1])
    history = model.fit(
        train_gen, validation_data=valid_gen, 
        steps_per_epoch=get_steps_per_epoch(int(0.9*(500*200)), batch_size), 
        validation_steps=get_steps_per_epoch(int(0.1*(500*200)), batch_size),
        epochs=n_epochs, callbacks=[es_callback, lr_callback]
    )
    model_path = name+str(n_epochs)+'.h5'
    print("Saving model to:",model_path)
    if not os.path.exists('models'):
        os.mkdir("models")
    model.save(os.path.join('models',model_path))
    # Evaluate the model
    print("Evaluating model:",name[:-1])
    test_res = model.evaluate(test_gen, steps=get_steps_per_epoch(500*50, batch_size))
    
    # Print the results as a dictionary {<metric name>: <value>}
    test_res_dict = dict(zip(model.metrics_names, test_res))
    print(test_res_dict)

Try for 15 epochs

In [7]:
n_epochs = 15
train_test_model(vgg16_model,"vgg_16_",n_epochs)

Training model: vgg_16
Epoch 1/15
2813/2813 [==============================] - 442s 156ms/step - loss: 97.8764 - accuracy: 0.1383 - val_loss: 112.4329 - val_accuracy: 0.1832 - lr: 0.0010
Epoch 2/15
2813/2813 [==============================] - 461s 164ms/step - loss: 116.5817 - accuracy: 0.2059 - val_loss: 131.1448 - val_accuracy: 0.2017 - lr: 0.0010
Epoch 3/15
2813/2813 [==============================] - 422s 150ms/step - loss: 125.0914 - accuracy: 0.2297 - val_loss: 135.1010 - val_accuracy: 0.2276 - lr: 0.0010
Epoch 4/15
2813/2813 [==============================] - 421s 150ms/step - loss: 129.5921 - accuracy: 0.2483 - val_loss: 146.8262 - val_accuracy: 0.2230 - lr: 0.0010
Epoch 5/15
2813/2813 [==============================] - 419s 149ms/step - loss: 133.8336 - accuracy: 0.2599 - val_loss: 149.8744 - val_accuracy: 0.2436 - lr: 0.0010
Epoch 6/15
2813/2813 [==============================] - ETA: 0s - loss: 137.4862 - accuracy: 0.2699
Epoch 6: ReduceLROnPlateau reducing learning rate to 

In [8]:
train_test_model(vgg19_model,"vgg_19_",n_epochs)

Training model: vgg_19
Epoch 1/15
2813/2813 [==============================] - 423s 150ms/step - loss: 89.9761 - accuracy: 0.1363 - val_loss: 105.4751 - val_accuracy: 0.1757 - lr: 0.0010
Epoch 2/15
2813/2813 [==============================] - 424s 151ms/step - loss: 106.7084 - accuracy: 0.2005 - val_loss: 118.8325 - val_accuracy: 0.2022 - lr: 0.0010
Epoch 3/15
2813/2813 [==============================] - 426s 152ms/step - loss: 113.7364 - accuracy: 0.2267 - val_loss: 127.1895 - val_accuracy: 0.2116 - lr: 0.0010
Epoch 4/15
2813/2813 [==============================] - 423s 150ms/step - loss: 118.7862 - accuracy: 0.2430 - val_loss: 133.5663 - val_accuracy: 0.2224 - lr: 0.0010
Epoch 5/15
2813/2813 [==============================] - 425s 151ms/step - loss: 122.4207 - accuracy: 0.2540 - val_loss: 139.5491 - val_accuracy: 0.2309 - lr: 0.0010
Epoch 6/15
2813/2813 [==============================] - ETA: 0s - loss: 125.5478 - accuracy: 0.2631
Epoch 6: ReduceLROnPlateau reducing learning rate to 

In [9]:
train_test_model(resnet50v2_model,"resnet_50_",n_epochs)

Training model: resnet_50
Epoch 1/15
2813/2813 [==============================] - 427s 151ms/step - loss: 73.3253 - accuracy: 0.0219 - val_loss: 71.3381 - val_accuracy: 0.0264 - lr: 0.0010
Epoch 2/15
2813/2813 [==============================] - 424s 151ms/step - loss: 71.4292 - accuracy: 0.0312 - val_loss: 72.6323 - val_accuracy: 0.0264 - lr: 0.0010
Epoch 3/15
2813/2813 [==============================] - 428s 152ms/step - loss: 69.1710 - accuracy: 0.0358 - val_loss: 72.6860 - val_accuracy: 0.0305 - lr: 0.0010
Epoch 4/15
2813/2813 [==============================] - 421s 150ms/step - loss: 68.9773 - accuracy: 0.0384 - val_loss: 69.1739 - val_accuracy: 0.0322 - lr: 0.0010
Epoch 5/15
2813/2813 [==============================] - 424s 151ms/step - loss: 68.9473 - accuracy: 0.0402 - val_loss: 76.7712 - val_accuracy: 0.0370 - lr: 0.0010
Epoch 6/15
2813/2813 [==============================] - 420s 149ms/step - loss: 67.9721 - accuracy: 0.0419 - val_loss: 65.8077 - val_accuracy: 0.0397 - lr: 0.0

In [10]:
train_test_model(resnet101V2_model,"resnet_101_",n_epochs)

Training model: resnet_101
Epoch 1/15
2813/2813 [==============================] - 428s 151ms/step - loss: 182.6077 - accuracy: 0.0156 - val_loss: 148.2959 - val_accuracy: 0.0214 - lr: 0.0010
Epoch 2/15
2813/2813 [==============================] - 427s 152ms/step - loss: 178.1198 - accuracy: 0.0212 - val_loss: 169.0116 - val_accuracy: 0.0216 - lr: 0.0010
Epoch 3/15
2813/2813 [==============================] - 425s 151ms/step - loss: 176.5454 - accuracy: 0.0242 - val_loss: 171.3275 - val_accuracy: 0.0324 - lr: 0.0010
Epoch 4/15
2813/2813 [==============================] - 427s 152ms/step - loss: 176.1174 - accuracy: 0.0261 - val_loss: 192.0960 - val_accuracy: 0.0219 - lr: 0.0010
Epoch 5/15
2813/2813 [==============================] - 427s 152ms/step - loss: 171.7699 - accuracy: 0.0276 - val_loss: 177.9083 - val_accuracy: 0.0206 - lr: 0.0010
Epoch 6/15
2813/2813 [==============================] - ETA: 0s - loss: 173.0019 - accuracy: 0.0293
Epoch 6: ReduceLROnPlateau reducing learning rat

In [11]:
train_test_model(inceptionv3_model,"inception_v3",n_epochs)

Training model: inception_v
Epoch 1/15
2813/2813 [==============================] - 426s 151ms/step - loss: 30.4384 - accuracy: 0.0157 - val_loss: 29.3523 - val_accuracy: 0.0182 - lr: 0.0010
Epoch 2/15
2813/2813 [==============================] - 423s 150ms/step - loss: 28.9521 - accuracy: 0.0190 - val_loss: 31.1039 - val_accuracy: 0.0202 - lr: 0.0010
Epoch 3/15
2813/2813 [==============================] - 430s 153ms/step - loss: 28.4059 - accuracy: 0.0209 - val_loss: 29.9128 - val_accuracy: 0.0191 - lr: 0.0010
Epoch 4/15
2813/2813 [==============================] - 423s 150ms/step - loss: 28.2938 - accuracy: 0.0219 - val_loss: 29.6297 - val_accuracy: 0.0239 - lr: 0.0010
Epoch 5/15
2813/2813 [==============================] - 422s 150ms/step - loss: 28.1465 - accuracy: 0.0233 - val_loss: 28.7201 - val_accuracy: 0.0213 - lr: 0.0010
Epoch 6/15
2813/2813 [==============================] - 422s 150ms/step - loss: 28.5179 - accuracy: 0.0233 - val_loss: 30.9236 - val_accuracy: 0.0259 - lr: 0

Try for 50 epochs

In [12]:
n_epochs = 35
# the for loop is prone to OOM errors
# calling training for each model individually
# for (model, name) in models:
#     print("Training model:",name[:-1])
#     train_test_model(model,name,n_epochs)
train_test_model(vgg16_model,"vgg_16_",n_epochs)

Training model: vgg_16
Epoch 1/35
2813/2813 [==============================] - 422s 150ms/step - loss: 77.3275 - accuracy: 0.3620 - val_loss: 99.6394 - val_accuracy: 0.2962 - lr: 1.0000e-04
Epoch 2/35
2813/2813 [==============================] - 420s 149ms/step - loss: 74.9489 - accuracy: 0.3658 - val_loss: 99.6033 - val_accuracy: 0.2966 - lr: 1.0000e-04
Epoch 3/35
2813/2813 [==============================] - 420s 149ms/step - loss: 73.5450 - accuracy: 0.3667 - val_loss: 96.4985 - val_accuracy: 0.2910 - lr: 1.0000e-04
Epoch 4/35
2813/2813 [==============================] - 420s 149ms/step - loss: 72.1212 - accuracy: 0.3685 - val_loss: 95.4666 - val_accuracy: 0.2986 - lr: 1.0000e-04
Epoch 5/35
2813/2813 [==============================] - 421s 150ms/step - loss: 70.5878 - accuracy: 0.3695 - val_loss: 93.9825 - val_accuracy: 0.2982 - lr: 1.0000e-04
Epoch 6/35
2813/2813 [==============================] - 422s 150ms/step - loss: 69.3428 - accuracy: 0.3716 - val_loss: 93.0719 - val_accuracy:

In [13]:
train_test_model(vgg19_model,"vgg_19_",n_epochs)

Training model: vgg_19
Epoch 1/35
2813/2813 [==============================] - 423s 150ms/step - loss: 68.9714 - accuracy: 0.3562 - val_loss: 89.2253 - val_accuracy: 0.2815 - lr: 1.0000e-04
Epoch 2/35
2813/2813 [==============================] - 422s 150ms/step - loss: 67.5415 - accuracy: 0.3574 - val_loss: 87.9904 - val_accuracy: 0.2918 - lr: 1.0000e-04
Epoch 3/35
2813/2813 [==============================] - 424s 151ms/step - loss: 65.9360 - accuracy: 0.3585 - val_loss: 86.4893 - val_accuracy: 0.2921 - lr: 1.0000e-04
Epoch 4/35
2813/2813 [==============================] - 423s 150ms/step - loss: 65.0243 - accuracy: 0.3600 - val_loss: 84.3622 - val_accuracy: 0.2904 - lr: 1.0000e-04
Epoch 5/35
2813/2813 [==============================] - 423s 150ms/step - loss: 63.8752 - accuracy: 0.3622 - val_loss: 83.1889 - val_accuracy: 0.2909 - lr: 1.0000e-04
Epoch 6/35
2813/2813 [==============================] - 424s 151ms/step - loss: 62.2325 - accuracy: 0.3626 - val_loss: 82.4855 - val_accuracy:

In [14]:
train_test_model(resnet50v2_model,"resnet_50_",n_epochs)

Training model: resnet_50
Epoch 1/35
2813/2813 [==============================] - 426s 152ms/step - loss: 66.5302 - accuracy: 0.0509 - val_loss: 58.4711 - val_accuracy: 0.0515 - lr: 0.0010
Epoch 2/35
2813/2813 [==============================] - 428s 152ms/step - loss: 65.4454 - accuracy: 0.0523 - val_loss: 68.7339 - val_accuracy: 0.0433 - lr: 0.0010
Epoch 3/35
2813/2813 [==============================] - 426s 151ms/step - loss: 66.3366 - accuracy: 0.0530 - val_loss: 72.9455 - val_accuracy: 0.0516 - lr: 0.0010
Epoch 4/35
2813/2813 [==============================] - 424s 151ms/step - loss: 65.7546 - accuracy: 0.0550 - val_loss: 57.0018 - val_accuracy: 0.0562 - lr: 0.0010
Epoch 5/35
2813/2813 [==============================] - 425s 151ms/step - loss: 65.7521 - accuracy: 0.0547 - val_loss: 73.8647 - val_accuracy: 0.0455 - lr: 0.0010
Epoch 6/35
2813/2813 [==============================] - 426s 152ms/step - loss: 65.7521 - accuracy: 0.0548 - val_loss: 70.9989 - val_accuracy: 0.0444 - lr: 0.0

In [16]:
train_test_model(resnet101V2_model,"resnet_101_",n_epochs)

Training model: resnet_101
Epoch 1/35
2813/2813 [==============================] - 433s 154ms/step - loss: 7.8578 - accuracy: 0.0837 - val_loss: 8.1302 - val_accuracy: 0.0787 - lr: 1.0000e-05
Epoch 2/35
2813/2813 [==============================] - 429s 152ms/step - loss: 7.8141 - accuracy: 0.0836 - val_loss: 8.2030 - val_accuracy: 0.0766 - lr: 1.0000e-05
Epoch 3/35
2813/2813 [==============================] - 427s 152ms/step - loss: 7.8003 - accuracy: 0.0845 - val_loss: 8.2044 - val_accuracy: 0.0774 - lr: 1.0000e-05
Epoch 4/35
2813/2813 [==============================] - 429s 152ms/step - loss: 7.7900 - accuracy: 0.0842 - val_loss: 8.2821 - val_accuracy: 0.0744 - lr: 1.0000e-05
Epoch 5/35
2813/2813 [==============================] - 429s 153ms/step - loss: 7.7505 - accuracy: 0.0827 - val_loss: 8.1566 - val_accuracy: 0.0773 - lr: 1.0000e-05
Epoch 6/35
2813/2813 [==============================] - ETA: 0s - loss: 7.7357 - accuracy: 0.0844
Epoch 6: ReduceLROnPlateau reducing learning rate 

In [17]:
train_test_model(inceptionv3_model,"inception_v3",n_epochs)

Training model: inception_v
Epoch 1/35
2813/2813 [==============================] - 427s 152ms/step - loss: 28.1324 - accuracy: 0.0259 - val_loss: 27.1632 - val_accuracy: 0.0290 - lr: 0.0010
Epoch 2/35
2813/2813 [==============================] - 427s 152ms/step - loss: 27.9710 - accuracy: 0.0270 - val_loss: 28.2413 - val_accuracy: 0.0244 - lr: 0.0010
Epoch 3/35
2813/2813 [==============================] - 429s 152ms/step - loss: 28.1690 - accuracy: 0.0263 - val_loss: 29.4570 - val_accuracy: 0.0227 - lr: 0.0010
Epoch 4/35
2813/2813 [==============================] - 454s 161ms/step - loss: 28.0276 - accuracy: 0.0266 - val_loss: 29.6164 - val_accuracy: 0.0276 - lr: 0.0010
Epoch 5/35
2813/2813 [==============================] - 427s 152ms/step - loss: 28.1346 - accuracy: 0.0270 - val_loss: 25.9030 - val_accuracy: 0.0286 - lr: 0.0010
Epoch 6/35
2813/2813 [==============================] - 428s 152ms/step - loss: 28.1860 - accuracy: 0.0275 - val_loss: 27.8177 - val_accuracy: 0.0249 - lr: 0

Finetuning is taking much longer than I hoped. 100 epochs is beyond reasonable execution time. I'll do 20 more epochs on each model to bring the total number of epochs trained to 70

In [18]:
n_epochs = 20
train_test_model(vgg16_model,"vgg_16_",n_epochs)

Training model: vgg_16
Epoch 1/20
2813/2813 [==============================] - 427s 152ms/step - loss: 47.1535 - accuracy: 0.3918 - val_loss: 69.8604 - val_accuracy: 0.2949 - lr: 1.0000e-04
Epoch 2/20
2813/2813 [==============================] - 429s 152ms/step - loss: 47.0564 - accuracy: 0.3903 - val_loss: 67.6218 - val_accuracy: 0.2991 - lr: 1.0000e-04
Epoch 3/20
2813/2813 [==============================] - 429s 153ms/step - loss: 46.2903 - accuracy: 0.3935 - val_loss: 67.4001 - val_accuracy: 0.2954 - lr: 1.0000e-04
Epoch 4/20
2813/2813 [==============================] - 429s 152ms/step - loss: 45.9570 - accuracy: 0.3970 - val_loss: 67.8367 - val_accuracy: 0.3019 - lr: 1.0000e-04
Epoch 5/20
2813/2813 [==============================] - 431s 153ms/step - loss: 45.5338 - accuracy: 0.3931 - val_loss: 66.7815 - val_accuracy: 0.3010 - lr: 1.0000e-04
Epoch 6/20
2813/2813 [==============================] - 429s 153ms/step - loss: 45.2008 - accuracy: 0.3964 - val_loss: 65.9101 - val_accuracy:

In [19]:
train_test_model(vgg19_model,"vgg_19_",n_epochs)

Training model: vgg_19
Epoch 1/20
2813/2813 [==============================] - 428s 152ms/step - loss: 41.8585 - accuracy: 0.3839 - val_loss: 59.8364 - val_accuracy: 0.2986 - lr: 1.0000e-04
Epoch 2/20
2813/2813 [==============================] - 429s 152ms/step - loss: 41.7384 - accuracy: 0.3844 - val_loss: 59.8510 - val_accuracy: 0.2975 - lr: 1.0000e-04
Epoch 3/20
2813/2813 [==============================] - 429s 152ms/step - loss: 41.1127 - accuracy: 0.3856 - val_loss: 60.0446 - val_accuracy: 0.2869 - lr: 1.0000e-04
Epoch 4/20
2813/2813 [==============================] - 433s 154ms/step - loss: 40.9588 - accuracy: 0.3827 - val_loss: 59.7701 - val_accuracy: 0.2897 - lr: 1.0000e-04
Epoch 5/20
2813/2813 [==============================] - 427s 152ms/step - loss: 40.4126 - accuracy: 0.3855 - val_loss: 58.6765 - val_accuracy: 0.2968 - lr: 1.0000e-04
Epoch 6/20
2813/2813 [==============================] - 431s 153ms/step - loss: 40.3058 - accuracy: 0.3854 - val_loss: 58.5677 - val_accuracy:

In [20]:
train_test_model(resnet50v2_model,"resnet_50_",n_epochs)

Training model: resnet_50
Epoch 1/20
2813/2813 [==============================] - 432s 154ms/step - loss: 9.9707 - accuracy: 0.1104 - val_loss: 10.6205 - val_accuracy: 0.0965 - lr: 1.0000e-04
Epoch 2/20
2813/2813 [==============================] - 431s 153ms/step - loss: 9.8960 - accuracy: 0.1108 - val_loss: 10.5721 - val_accuracy: 0.0929 - lr: 1.0000e-04
Epoch 3/20
2813/2813 [==============================] - 441s 157ms/step - loss: 9.8099 - accuracy: 0.1129 - val_loss: 10.2911 - val_accuracy: 0.0905 - lr: 1.0000e-04
Epoch 4/20
2813/2813 [==============================] - 430s 153ms/step - loss: 9.7888 - accuracy: 0.1098 - val_loss: 11.0859 - val_accuracy: 0.0885 - lr: 1.0000e-04
Epoch 5/20
2813/2813 [==============================] - 430s 153ms/step - loss: 9.7174 - accuracy: 0.1114 - val_loss: 10.3458 - val_accuracy: 0.0916 - lr: 1.0000e-04
Epoch 6/20
2813/2813 [==============================] - 431s 153ms/step - loss: 9.6192 - accuracy: 0.1107 - val_loss: 10.9007 - val_accuracy: 0.

In [21]:
train_test_model(resnet101V2_model,"resnet_101_",n_epochs)

Training model: resnet_101
Epoch 1/20
2813/2813 [==============================] - 433s 154ms/step - loss: 7.0840 - accuracy: 0.0914 - val_loss: 7.4825 - val_accuracy: 0.0809 - lr: 1.0000e-07
Epoch 2/20
2813/2813 [==============================] - 431s 153ms/step - loss: 7.0416 - accuracy: 0.0930 - val_loss: 7.5009 - val_accuracy: 0.0842 - lr: 1.0000e-07
Epoch 3/20
2813/2813 [==============================] - 469s 167ms/step - loss: 7.0659 - accuracy: 0.0927 - val_loss: 7.4887 - val_accuracy: 0.0851 - lr: 1.0000e-07
Epoch 4/20
2813/2813 [==============================] - 431s 153ms/step - loss: 7.0593 - accuracy: 0.0910 - val_loss: 7.4320 - val_accuracy: 0.0769 - lr: 1.0000e-07
Epoch 5/20
2813/2813 [==============================] - 432s 154ms/step - loss: 7.0739 - accuracy: 0.0921 - val_loss: 7.4820 - val_accuracy: 0.0795 - lr: 1.0000e-07
Epoch 6/20
2813/2813 [==============================] - 434s 154ms/step - loss: 7.0293 - accuracy: 0.0930 - val_loss: 7.5482 - val_accuracy: 0.0841 

In [22]:
train_test_model(inceptionv3_model,"inception_v3",n_epochs)

Training model: inception_v
Epoch 1/20
2813/2813 [==============================] - 433s 154ms/step - loss: 6.1865 - accuracy: 0.0541 - val_loss: 6.5046 - val_accuracy: 0.0470 - lr: 1.0000e-04
Epoch 2/20
2813/2813 [==============================] - 430s 153ms/step - loss: 6.1643 - accuracy: 0.0544 - val_loss: 6.3543 - val_accuracy: 0.0482 - lr: 1.0000e-04
Epoch 3/20
2813/2813 [==============================] - 431s 153ms/step - loss: 6.1505 - accuracy: 0.0532 - val_loss: 6.2087 - val_accuracy: 0.0473 - lr: 1.0000e-04
Epoch 4/20
2813/2813 [==============================] - 432s 154ms/step - loss: 6.1391 - accuracy: 0.0540 - val_loss: 6.4592 - val_accuracy: 0.0460 - lr: 1.0000e-04
Epoch 5/20
2813/2813 [==============================] - 431s 153ms/step - loss: 6.1394 - accuracy: 0.0535 - val_loss: 6.2819 - val_accuracy: 0.0495 - lr: 1.0000e-04
Epoch 6/20
2813/2813 [==============================] - 431s 153ms/step - loss: 6.1117 - accuracy: 0.0539 - val_loss: 6.3007 - val_accuracy: 0.0469

Here are some summary of the stuctures of the models we just trained to help my analysis

In [23]:
# for analysis propurses
print("VGG16")
vgg16_model.summary()
print("VGG19")
vgg19_model.summary()
print("Resnet50")
resnet50v2_model.summary()
print("Resnet100")
resnet101V2_model.summary()
print("InceptionV3")
inceptionv3_model.summary()

VGG16
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)     